In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import numpy as np
import re

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

In [3]:
df= pd.read_excel("Resources/SanctionedAthletes.xlsx")
#application_df.head()
df = df.rename(columns={ 'ID': 'AthleteID'})
df = df.drop(columns={'Unnamed: 0'})
df = df.drop(columns={'Name_x','AthleteName','FirstName','LastName','Age','Name_y','Role','Discipline1','Discipline2','InfractionType','ADRVRules','ADRVNotes','Description'})
#'Sanction','PeriodofDisqualificationOfResults'
df

,AthleteID,Sex,NOC,DateofBirth,DateofInfraction,Sanction,IneligibilityUntil,PeriodofDisqualificationOfResults,LifetimeBan
0,22342,M,USA,1996-03-06,2019-12-09,18 months ineligibility,2021-11-13,Since 09.12.19,No
1,30123,M,USA,1968-09-09,NaT,8 years ineligibility,2022-12-16,NaN,No
2,133214,M,USA,1976-08-14,2004-07-23,Lifetime ban (2nd ADRV),NaT,Since 23.07-04,Yes
3,47498,F,ROU,1978-10-01,2014-02-07,8 years ineligibility (2nd ADRV),2022-03-10,2014-02-07 00:00:00,No
4,25370,M,FRA,1987-12-22,2020-01-31,16 months ineligibility,2021-05-31,Since 31.01.20,No
...,...,...,...,...,...,...,...,...,...
68,20812,M,ZAM,1988-11-12,2017-06-17,4 years ineligibility,2021-08-14,Since 17.06.17,No
69,109177,F,ALB,1979-08-22,2012-07-31,2 years ineligibility,2022-02-02,From 31.07.12 to 30.07.14,No
70,54211,F,BOT,1990-06-22,2017-10-14,4 years ineligibility,2021-10-13,Since 14/10/2017,No
71,83397,F,ZAM,1992-09-21,2017-08-06,4 years ineligibility,2021-11-08,Since 06.08.17,No


In [4]:
df.loc[df['Sanction'].isnull(),'Sanction'] = '0'
df[df['DateofInfraction'].isnull() | df['IneligibilityUntil'].isnull()]


,AthleteID,Sex,NOC,DateofBirth,DateofInfraction,Sanction,IneligibilityUntil,PeriodofDisqualificationOfResults,LifetimeBan
1,30123,M,USA,1968-09-09,NaT,8 years ineligibility,2022-12-16,NaN,No
2,133214,M,USA,1976-08-14,2004-07-23,Lifetime ban (2nd ADRV),NaT,Since 23.07-04,Yes
6,20486,F,RUS,1988-01-29,2013-06-30,0,NaT,From 21.07.15 to 05.02.16,No
7,59178,F,RUS,1982-06-04,NaT,8 years ineligibility,2025-01-05,From 15.07.12 - 06.08.14,No
9,72681,F,RUS,1982-06-04,NaT,8 years ineligibility,2024-07-01,From 16.07.12 to 02.07.16,No
13,89225,F,ESP,1986-07-15,2015-08-21,Lifetime ban (3rd ADRV),NaT,Since 01.08.15,Yes
14,3631,M,BLR,1985-11-29,2012-08-08,0,NaT,From 08.08.12 to 27.05.15,No
32,132898,M,RUS,1986-09-04,2011-02-25,Lifetime ban as from 25/02/2011,NaT,NaN,Yes
33,90825,M,BRA,1977-06-22,2009-04-18,Lifetime ban,NaT,Since 18.04.09,Yes
35,123575,F,TUR,1985-03-12,2011-08-30,Lifetime ban,NaT,Since 30-08-11,Yes


In [5]:
df[df['DateofInfraction'].isnull()]

,AthleteID,Sex,NOC,DateofBirth,DateofInfraction,Sanction,IneligibilityUntil,PeriodofDisqualificationOfResults,LifetimeBan
1,30123,M,USA,1968-09-09,NaT,8 years ineligibility,2022-12-16,NaN,No
7,59178,F,RUS,1982-06-04,NaT,8 years ineligibility,2025-01-05,From 15.07.12 - 06.08.14,No
9,72681,F,RUS,1982-06-04,NaT,8 years ineligibility,2024-07-01,From 16.07.12 to 02.07.16,No


In [6]:
df.dtypes

AthleteID                                     int64
Sex                                          object
NOC                                          object
DateofBirth                          datetime64[ns]
DateofInfraction                     datetime64[ns]
Sanction                                     object
IneligibilityUntil                   datetime64[ns]
PeriodofDisqualificationOfResults            object
LifetimeBan                                  object
dtype: object

In [7]:
df.loc[df['Sanction'] == '8 years ineligibility','DateofInfraction'] = df['IneligibilityUntil'] - pd.DateOffset(years=8)

In [8]:
df.head(8)

,AthleteID,Sex,NOC,DateofBirth,DateofInfraction,Sanction,IneligibilityUntil,PeriodofDisqualificationOfResults,LifetimeBan
0,22342,M,USA,1996-03-06,2019-12-09,18 months ineligibility,2021-11-13,Since 09.12.19,No
1,30123,M,USA,1968-09-09,2014-12-16,8 years ineligibility,2022-12-16,NaN,No
2,133214,M,USA,1976-08-14,2004-07-23,Lifetime ban (2nd ADRV),NaT,Since 23.07-04,Yes
3,47498,F,ROU,1978-10-01,2014-02-07,8 years ineligibility (2nd ADRV),2022-03-10,2014-02-07 00:00:00,No
4,25370,M,FRA,1987-12-22,2020-01-31,16 months ineligibility,2021-05-31,Since 31.01.20,No
5,107891,M,ITA,1984-12-26,2016-01-01,8 years ineligibility (2nd ADRV),2024-07-07,Since 01.01.16,No
6,20486,F,RUS,1988-01-29,2013-06-30,0,NaT,From 21.07.15 to 05.02.16,No
7,59178,F,RUS,1982-06-04,2017-01-05,8 years ineligibility,2025-01-05,From 15.07.12 - 06.08.14,No


In [9]:
df[df['IneligibilityUntil'].isnull()]

,AthleteID,Sex,NOC,DateofBirth,DateofInfraction,Sanction,IneligibilityUntil,PeriodofDisqualificationOfResults,LifetimeBan
2,133214,M,USA,1976-08-14,2004-07-23,Lifetime ban (2nd ADRV),NaT,Since 23.07-04,Yes
6,20486,F,RUS,1988-01-29,2013-06-30,0,NaT,From 21.07.15 to 05.02.16,No
13,89225,F,ESP,1986-07-15,2015-08-21,Lifetime ban (3rd ADRV),NaT,Since 01.08.15,Yes
14,3631,M,BLR,1985-11-29,2012-08-08,0,NaT,From 08.08.12 to 27.05.15,No
32,132898,M,RUS,1986-09-04,2011-02-25,Lifetime ban as from 25/02/2011,NaT,NaN,Yes
33,90825,M,BRA,1977-06-22,2009-04-18,Lifetime ban,NaT,Since 18.04.09,Yes
35,123575,F,TUR,1985-03-12,2011-08-30,Lifetime ban,NaT,Since 30-08-11,Yes
38,21407,F,NGR,1975-05-04,2012-06-21,Lifetime ban as from 12/07/2012,NaT,Since 21.06.12,Yes
40,95198,M,UKR,1978-06-07,2005-08-08,Lifetime ban (2nd ADRV),NaT,Since 08.08.05,Yes
64,69674,M,AUT,1974-05-25,2007-11-20,Lifetime ban (2nd ADRV),NaT,Since 20.11.07,Yes


In [10]:
df.loc[df['Sanction'].str.startswith('Lifetime ban', na=False),'IneligibilityUntil'] = pd.Timestamp.max
df[df['DateofInfraction'].isnull() | df['IneligibilityUntil'].isnull()]

,AthleteID,Sex,NOC,DateofBirth,DateofInfraction,Sanction,IneligibilityUntil,PeriodofDisqualificationOfResults,LifetimeBan
6,20486,F,RUS,1988-01-29,2013-06-30,0,NaT,From 21.07.15 to 05.02.16,No
14,3631,M,BLR,1985-11-29,2012-08-08,0,NaT,From 08.08.12 to 27.05.15,No


In [11]:
m = df['IneligibilityUntil'].isnull()
df.loc[m,'IneligibilityUntil'] = pd.to_datetime(df.loc[m,'PeriodofDisqualificationOfResults'].astype(str).replace(r'^.+(\d+)\.(\d+)\.(\d+)$',r'\1-\2-\3',regex=True))


In [12]:
df[df['DateofInfraction'].isnull() | df['IneligibilityUntil'].isnull()]

,AthleteID,Sex,NOC,DateofBirth,DateofInfraction,Sanction,IneligibilityUntil,PeriodofDisqualificationOfResults,LifetimeBan


In [13]:
df.dtypes

AthleteID                                     int64
Sex                                          object
NOC                                          object
DateofBirth                          datetime64[ns]
DateofInfraction                     datetime64[ns]
Sanction                                     object
IneligibilityUntil                   datetime64[ns]
PeriodofDisqualificationOfResults            object
LifetimeBan                                  object
dtype: object

In [14]:
df.head(7)

,AthleteID,Sex,NOC,DateofBirth,DateofInfraction,Sanction,IneligibilityUntil,PeriodofDisqualificationOfResults,LifetimeBan
0,22342,M,USA,1996-03-06,2019-12-09,18 months ineligibility,2021-11-13 00:00:00.000000000,Since 09.12.19,No
1,30123,M,USA,1968-09-09,2014-12-16,8 years ineligibility,2022-12-16 00:00:00.000000000,NaN,No
2,133214,M,USA,1976-08-14,2004-07-23,Lifetime ban (2nd ADRV),2262-04-11 23:47:16.854775807,Since 23.07-04,Yes
3,47498,F,ROU,1978-10-01,2014-02-07,8 years ineligibility (2nd ADRV),2022-03-10 00:00:00.000000000,2014-02-07 00:00:00,No
4,25370,M,FRA,1987-12-22,2020-01-31,16 months ineligibility,2021-05-31 00:00:00.000000000,Since 31.01.20,No
5,107891,M,ITA,1984-12-26,2016-01-01,8 years ineligibility (2nd ADRV),2024-07-07 00:00:00.000000000,Since 01.01.16,No
6,20486,F,RUS,1988-01-29,2013-06-30,0,2016-05-02 00:00:00.000000000,From 21.07.15 to 05.02.16,No


In [15]:
df['LifetimeBan'] = np.where(df['LifetimeBan'] == 'LifetimeBan', 1, 0)
df['Sex'] = np.where(df['Sex'] == 'M', 1, 0)
df = df.drop(columns={'Sanction','PeriodofDisqualificationOfResults','NOC'})
df['IneligibilityUntil'] = pd.DatetimeIndex(df['IneligibilityUntil']).year
df['DateofInfraction'] = pd.DatetimeIndex(df['DateofInfraction']).year
df['DateofBirth'] = pd.DatetimeIndex(df['DateofBirth']).year
df.head(7)

,AthleteID,Sex,DateofBirth,DateofInfraction,IneligibilityUntil,LifetimeBan
0,22342,1,1996,2019,2021,0
1,30123,1,1968,2014,2022,0
2,133214,1,1976,2004,2262,0
3,47498,0,1978,2014,2022,0
4,25370,1,1987,2020,2021,0
5,107891,1,1984,2016,2024,0
6,20486,0,1988,2013,2016,0


In [16]:
#use athlete data of athletes peforming since 2004 for the olympics
df.DateofInfraction.min() 

2004

In [17]:
# Load the data
file_path = 'Resources/cleaned_athlete_data.csv'
df_athlete_data = pd.read_csv(file_path)
df_athlete_data

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal_Type,Medal,Row
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,No_Medal,N,2
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,No_Medal,N,3
2,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,No_Medal,N,4
3,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",No_Medal,N,5
4,5,Christine Jacoba Aaftink,F,25.0,185.0,82.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,Speed Skating Women's 500 metres,No_Medal,N,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188041,135569,Andrzej ya,M,29.0,179.0,89.0,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,No_Medal,N,199834
188042,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",No_Medal,N,199835
188043,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",No_Medal,N,199836
188044,135571,Tomasz Ireneusz ya,M,30.0,185.0,96.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,No_Medal,N,199837


In [18]:
indexNames = df_athlete_data[ df_athlete_data['Year'] <= 2004].index
df_athlete_data['Sex'] = np.where(df_athlete_data['Sex'] == 'M', 1, 0)
df_athlete_data['Medal'] = np.where(df_athlete_data['Medal'] == 'Y', 1, 0)
df_athlete_data = df_athlete_data.rename(columns={ 'ID': 'AthleteID'})
df_athlete_data.drop(indexNames , inplace=True)
df_athlete_data

,AthleteID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal_Type,Medal,Row
1,2,A Lamusi,1,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,No_Medal,0,3
30,16,Juhamatti Tapio Aaltonen,1,28.0,184.0,85.0,Finland,FIN,2014 Winter,2014,Winter,Sochi,Ice Hockey,Ice Hockey Men's Ice Hockey,Bronze,1,32
50,20,Kjetil Andr Aamodt,1,34.0,176.0,85.0,Norway,NOR,2006 Winter,2006,Winter,Torino,Alpine Skiing,Alpine Skiing Men's Downhill,No_Medal,0,52
51,20,Kjetil Andr Aamodt,1,34.0,176.0,85.0,Norway,NOR,2006 Winter,2006,Winter,Torino,Alpine Skiing,Alpine Skiing Men's Super G,Gold,1,53
52,22,Andreea Aanei,0,22.0,170.0,125.0,Romania,ROU,2016 Summer,2016,Summer,Rio de Janeiro,Weightlifting,Weightlifting Women's Super-Heavyweight,No_Medal,0,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188022,135558,ukasz Tomasz ygado,1,32.0,200.0,89.0,Poland,POL,2012 Summer,2012,Summer,London,Volleyball,Volleyball Men's Volleyball,No_Medal,0,199815
188029,135559,Pawe Jan Zygmunt,1,33.0,182.0,79.0,Poland,POL,2006 Winter,2006,Winter,Torino,Speed Skating,"Speed Skating Men's 5,000 metres",No_Medal,0,199822
188040,135568,Olga Igorevna Zyuzkova,0,33.0,171.0,69.0,Belarus,BLR,2016 Summer,2016,Summer,Rio de Janeiro,Basketball,Basketball Women's Basketball,No_Medal,0,199833
188042,135570,Piotr ya,1,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",No_Medal,0,199835


In [19]:
df_athlete_data.drop(columns = {'Name','Age','Height','Weight','Team','Games','Season','City','Sport','Event','Medal_Type','Row'}, axis=1, inplace=True)
#df_athlete_data.drop(columns = {'NOC'}, axis=1, inplace=True)
df_athlete_data

,AthleteID,Sex,NOC,Year,Medal
1,2,1,CHN,2012,0
30,16,1,FIN,2014,1
50,20,1,NOR,2006,0
51,20,1,NOR,2006,1
52,22,0,ROU,2016,0
...,...,...,...,...,...
188022,135558,1,POL,2012,0
188029,135559,1,POL,2006,0
188040,135568,0,BLR,2016,0
188042,135570,1,POL,2014,0


In [20]:
athlete_participation = pd.DataFrame(df_athlete_data.groupby(['AthleteID','Sex','NOC','Year'])['Medal'].count().reset_index(name="count"))
#athlete_participation = pd.DataFrame(df_athlete_data.groupby(['AthleteID','Sex','Year'])['Medal'].count().reset_index(name="count"))
athlete_participation

,AthleteID,Sex,NOC,Year,count
0,2,1,CHN,2012,1
1,16,1,FIN,2014,1
2,20,1,NOR,2006,2
3,22,0,ROU,2016,1
4,34,1,FRA,2012,1
...,...,...,...,...,...
39477,135557,1,POL,2012,1
39478,135558,1,POL,2012,1
39479,135559,1,POL,2006,1
39480,135568,0,BLR,2016,1


In [21]:
df['Sanctioned'] = "Y"
df.dtypes

AthleteID              int64
Sex                    int32
DateofBirth            int64
DateofInfraction       int64
IneligibilityUntil     int64
LifetimeBan            int32
Sanctioned            object
dtype: object

In [22]:
###### df_athlete_data.dtypes

In [23]:
new_len = round(len(athlete_participation)/len(df))
repeated = (pd.concat([df] * new_len)
              .reset_index()
              .drop(["index"], 1)
              .iloc[:len(athlete_participation)])

dfFull = athlete_participation.merge(df, on=['AthleteID','Sex'], how='left')
dfFull = dfFull.fillna(0)
dfFull

,AthleteID,Sex,NOC,Year,count,DateofBirth,DateofInfraction,IneligibilityUntil,LifetimeBan,Sanctioned
0,2,1,CHN,2012,1,0.0,0.0,0.0,0.0,0
1,16,1,FIN,2014,1,0.0,0.0,0.0,0.0,0
2,20,1,NOR,2006,2,0.0,0.0,0.0,0.0,0
3,22,0,ROU,2016,1,0.0,0.0,0.0,0.0,0
4,34,1,FRA,2012,1,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
39477,135557,1,POL,2012,1,0.0,0.0,0.0,0.0,0
39478,135558,1,POL,2012,1,0.0,0.0,0.0,0.0,0
39479,135559,1,POL,2006,1,0.0,0.0,0.0,0.0,0
39480,135568,0,BLR,2016,1,0.0,0.0,0.0,0.0,0


In [24]:
dfFull.dtypes

AthleteID               int64
Sex                     int64
NOC                    object
Year                    int64
count                   int64
DateofBirth           float64
DateofInfraction      float64
IneligibilityUntil    float64
LifetimeBan           float64
Sanctioned             object
dtype: object

In [25]:
# Generate our categorical variable list
dfFull_cat = dfFull.dtypes[dfFull.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
dfFull[dfFull_cat].nunique()

NOC           210
Sanctioned      2
dtype: int64

In [26]:
#df2 = df2.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')


In [27]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(dfFull[dfFull_cat].astype(str)))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(dfFull_cat)
encode_df.head()

,NOC_AFG,NOC_AHO,NOC_ALB,NOC_ALG,NOC_AND,NOC_ANG,NOC_ANT,NOC_ARG,NOC_ARM,NOC_ARU,...,NOC_UZB,NOC_VAN,NOC_VEN,NOC_VIE,NOC_VIN,NOC_YEM,NOC_ZAM,NOC_ZIM,Sanctioned_0,Sanctioned_Y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [28]:
# Merge one-hot encoded features and drop the originals
dfFull = dfFull.merge(encode_df,left_index=True, right_index=True)
dfFull = dfFull.drop(dfFull_cat,1)
dfFull.head()

,AthleteID,Sex,Year,count,DateofBirth,DateofInfraction,IneligibilityUntil,LifetimeBan,NOC_AFG,NOC_AHO,...,NOC_UZB,NOC_VAN,NOC_VEN,NOC_VIE,NOC_VIN,NOC_YEM,NOC_ZAM,NOC_ZIM,Sanctioned_0,Sanctioned_Y
0,2,1,2012,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,16,1,2014,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20,1,2006,2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,22,0,2016,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,34,1,2012,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [29]:
# Split our preprocessed data into our features and target arrays
y = dfFull["Sanctioned_Y"].values
X = dfFull.drop(["Sanctioned_0"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [30]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [31]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 1.000


In [32]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
926/926 [==============================] - 19s 11ms/step - loss: 0.0140 - accuracy: 0.9994
Epoch 2/50
926/926 [==============================] - 6s 6ms/step - loss: 1.1599e-04 - accuracy: 1.0000
Epoch 3/50
926/926 [==============================] - 10s 10ms/step - loss: 3.1268e-05 - accuracy: 1.0000
Epoch 4/50
926/926 [==============================] - 10s 10ms/step - loss: 1.2780e-05 - accuracy: 1.0000
Epoch 5/50
926/926 [==============================] - 6s 6ms/step - loss: 6.1670e-06 - accuracy: 1.0000
Epoch 6/50
926/926 [==============================] - 10s 10ms/step - loss: 3.2784e-06 - accuracy: 1.0000
Epoch 7/50
926/926 [==============================] - 10s 10ms/step - loss: 1.7951e-06 - accuracy: 1.0000
Epoch 8/50
926/926 [==============================] - 6s 6ms/step - loss: 1.0192e-06 - accuracy: 1.0000
Epoch 9/50
926/926 [==============================] - 10s 11ms/step - loss: 5.8854e-07 - accuracy: 1.0000
Epoch 10/50
926/926 [==============================] - 1

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

309/309 - 0s - loss: 9.3495e-11 - accuracy: 1.0000
Loss: 9.349460095009476e-11, Accuracy: 1.0


In [34]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
confusion_matrix(y_test, y_pred)

array([[9852,    0],
       [   0,   19]], dtype=int64)

In [35]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      9852
         1.0       1.00      1.00      1.00        19

    accuracy                           1.00      9871
   macro avg       1.00      1.00      1.00      9871
weighted avg       1.00      1.00      1.00      9871



In [36]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))
#While precision ("pre" column) and recall ("rec" column) are high for the majority class, 
#  precision is low for the minority class. (below .45)

                   pre       rec       spe        f1       geo       iba       sup

        0.0       1.00      1.00      1.00      1.00      1.00      1.00      9852
        1.0       1.00      1.00      1.00      1.00      1.00      1.00        19

avg / total       1.00      1.00      1.00      1.00      1.00      1.00      9871

